# Time series plotting around reservoirs

## To-dos

In [ ]:
func_dir = '/global/home/users/ann_scheliga/CYGNSS-GRACE/codebase/'

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt


import codebase

In [ ]:
grace_dict = codebase.load_data.load_GRACE()
res_shp = codebase.load_data.load_GRanD()

In [ ]:
def single_timeseries_CYGNSS_GRACE(dam_name,**plot_params):
    """
    Description
    -----------
    Hard-coded time series slicing for CYGNSS to start two months after the intersecting_timeframes() start date
    
    Inputs
    ------
    dam_name: name of a dam in the GRanD reservoir dataset
    subset_method : str
        default : 'polygon'
        accepted : 'polygon' , 'point'
        allows for updated polygon or older point-based subsetting
    **plot_params : dict
        dictionary of plot formatting options and labels
            Keys used: None, all passed to `three_part_timeseries`

    Outputs
    -------
    precip , fw , cmwe : Pandas Series
        Series used for plotting
        Respectively, precipitation, fractional water, TWS
    """
    global grace_dict, res_shp
    
    subset_gpd = codebase.area_subsets.check_for_multiple_dams(dam_name,res_shp)

    subset_bbox = subset_gpd.geometry.buffer(0).bounds
    fw_DA = codebase.load_data.load_CYGNSS_001_all_months(subset_bbox)
    fw_multi , fw_full = codebase.area_subsets.cygnss_shape_subset(
                         subset_gpd,
                         fw_DA,
                         buffer_val=0.1,
                         crs_code=4326)
    fw_binary = codebase.area_calcs.cygnss_convert_to_binary(fw_multi)
    fw_num = codebase.area_calcs.CYGNSS_001_areal_average(fw_binary)
    fw_full = pd.Series(data=fw_num, index=fw_full.index)

    cmwe_multi , mascon_ts , cmwe_full = codebase.area_subsets.grace_shape_subset(
                            subset_gpd,
                            grace_dict,
                            buffer_val=1)
    print(mascon_ts)

    fw , cmwe = codebase.time_series_calcs.intersecting_timeframes(fw_full,cmwe_full,buffer=0)
    fw = fw[2:]
    
    plot_params['data_labels'] = ['CYGNSS','GRACE-FO','']
    plot_params['x_label'] = 'Time'
    plot_params['y_labels'] = ['Fraction of surface water coverage', 'TWS water equivalence (cm)','']
    plot_params['line_fmt'] = ['b-','k-','k-']
    plot_params['figsize'] = (15,6)
    plot_params['grid'] = True
    plot_params['x_ticks'] = 'yearly'

    plt.rc('font', size=20)
    plt.rcParams['lines.linewidth'] = 2.5  
    codebase.plotting_functions.three_part_timeseries([fw , cmwe],**plot_params)

    return fw , cmwe

## Individual reservoirs

In [ ]:
res_oi = 'Xiaowan'
plot_params = {
    'title' : res_oi
}
xiaowan_fw_series , xiaowan_cmwe_series = single_timeseries_CYGNSS_GRACE(
    res_oi,
    **plot_params)

In [ ]:
res_oi = 'Nuozhadu'
plot_params = {
    'title' : res_oi
}
nuozhadu_fw_series , nuozhadu_cmwe_series = single_timeseries_CYGNSS_GRACE(
    res_oi,
    **plot_params)

## Surface water

In [ ]:
fig, axs = plt.subplots()

axs.plot(xiaowan_cmwe_series, 'k')
axs.plot(nuozhadu_cmwe_series , 'b')

## Older time series metrics from TSM class objects

In [ ]:
def series_to_TSM_object(input_series,*TSM_args,**TSM_kwargs):
    series_obj = input_series.astype(float)
    series_obj.index = list(map(codebase.time_series_calcs.toYearFraction,series_obj.index))
    TSM_obj = codebase.time_series_calcs.TimeSeriesMetrics(series_obj,*TSM_args,**TSM_kwargs)
    return TSM_obj

In [ ]:
def quick_quad_xcorr_figure(input3TSMs,**plot_params):
    fig , axs = plt.subplots(2,2,figsize=(16,16))
    
    #Seasonality plot
    input3TSMs[0].plot_seasonality(axs[0,0],color='k',label='GRACE-FO')
    input3TSMs[1].plot_seasonality(axs[0,0],color='b',label='CYGNSS')
#     input3TSMs[2].plot_seasonality(axs[0,0],color='c',label='IMERG')
    axs[0,0].set_ylabel('Normalized seasonality')
    
    short_axs_names = [tick.get_text()[:1] for tick in axs[0,0].get_xticklabels()]
    axs[0,0].set_xticklabels(short_axs_names)
    axs[0,0].set_ylim([-1.9,1.8])
    axs[0,0].grid()
    axs[0,0].legend(loc='upper center');
    
    #Anomalies plot
    input3TSMs[0].plot_anomalies(axs[0,1],color='k',label='GRACE-FO')
    input3TSMs[1].plot_anomalies(axs[0,1],color='b',label='CYGNSS')
#     input3TSMs[2].plot_anomalies(axs[0,1],color='c',label='IMERG')
    axs[0,1].set_ylabel('Normalized anomalies')
    axs[0,1].set_xticklabels(axs[0,1].get_xticklabels())
    axs[0,1].set_ylim([-2.2,3.2])
    axs[0,1].grid()
    axs[0,1].legend(loc='upper center');
    
    #Seasonality xcorr
    lag_x , lag_y = input3TSMs[1].cross_corr(input3TSMs[0],axs[1,0],ts_type='season',plot_on=False);
    axs[1,0].grid()
    axs[1,0].set_xticks(np.arange(-10,11,2))
    axs[1,0].set_ylabel('Linear correlation')
    axs[1,0].set_xlabel('Time lag (months)')
    
    #Anomalies xcorr
    lag_x , lag_y = input3TSMs[1].cross_corr(input3TSMs[0],axs[1,1],plot_on=False)
    axs[1,1].grid()
    axs[1,1].set_xticks(np.arange(-10,11,2))
    axs[1,1].set_ylabel('Linear correlation')
    axs[1,1].set_xlabel('Time lag (months)')
    
    plt.show()
# quick_quad_xcorr_figure([cmwe_ts,fw_ts,precip_ts])

In [ ]:
def series_to_xcorr_fig(fw_series,cmwe_series):
    fw_ts = series_to_TSM_object(fw_series,'CYGNSS',remove_seasonality=True,zero_start=False,start_month=10)
    cmwe_ts = series_to_TSM_object(cmwe_series,'GRACE-FO',remove_seasonality=True,zero_start=False,start_month=10)
    quick_quad_xcorr_figure([cmwe_ts,fw_ts])

In [ ]:
series_to_xcorr_fig(xiaowan_fw_series,xiaowan_cmwe_series)

In [ ]:
series_to_xcorr_fig(nuozhadu_fw_series,nuozhadu_cmwe_series)

In [ ]:
shifted_fw_ts = codebase.time_series_calcs.normalize(pd.DataFrame(data = fw_ts.ts_detrend.values[:-7], index = fw_ts.ts_detrend.index[7:]))

In [ ]:
fig , ax = plt.subplots()
cmwe_ts.plot_anomalies(ax,color='k',label='GRACE')
ax.plot(shifted_fw_ts,color='b',label='CYGNSS')
ax.legend();
ax.grid()
ax.set_xlabel('GRACE timestep')
ax.set_ylabel('Normalized anomalies')
# ax.set_title('Shifted (-7 months) CYGNSS anomalies \ to max correlation with GRACE');

In [ ]:
# Goal plot anomalies with time lag applied
fig , ax = plt.subplots()
fw_ts.plot_anomalies(ax,color='b',label='CYGNSS')

In [ ]:
fig , ax = plt.subplots()
cmwe_ts.plot_seasonality(ax,color='k',label='GRACE')
fw_ts.plot_seasonality(ax,color='b',label='CYGNSS')
precip_ts.plot_seasonality(ax,color='c',label='IMERG')
ax.set_ylabel('Normalized seasonality')
ax.set_xticklabels(ax.get_xticklabels(),rotation=-90)
ax.grid()
ax.legend();

In [ ]:
print(cmwe_ts.lintrend_metrics)
fig , ax = plt.subplots()
cmwe_ts.plot_anomalies(ax,color='k',label='GRACE')
fw_ts.plot_anomalies(ax,color='b',label='CYGNSS')
precip_ts.plot_anomalies(ax,color='c',label='IMERG')
ax.legend();
ax.grid()

In [ ]:
fig , ax = plt.subplots()
lag_x , lag_y = cmwe_ts.cross_corr(precip_ts,ax,ts_type='season',plot_on=True)
lag_x , lag_y = fw_ts.cross_corr(cmwe_ts,ax,ts_type='season',plot_on=True);
lag_x , lag_y = fw_ts.cross_corr(precip_ts,ax,ts_type='season',plot_on=True)

In [ ]:
lag_x , lag_y = cmwe_ts.cross_corr(precip_ts,plot_on=True)
lag_x , lag_y = fw_ts.cross_corr(cmwe_ts,plot_on=True)
lag_x , lag_y = fw_ts.cross_corr(precip_ts,plot_on=True)

## Overview Map

In [ ]:
basin_res = res_shp.loc[res_shp['MAIN_BASIN']=='Mekong',['RES_NAME','DAM_NAME','RIVER','ADMIN_UNIT','CAP_MCM','AREA_SKM','MAIN_USE','geometry']]
basin_res.explore(height=500,width=450,popup='DAM_NAME',column='CAP_MCM',cmap='winter_r',
                legend = False,scheme='NaturalBreaks')

## Scratchwork

In [ ]:
grace_dict['mascon']